<a href="https://colab.research.google.com/github/deayalar/deeplearning_unitn/blob/main/DL_Project_wandb_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://market1501.s3-us-west-2.amazonaws.com/dataset.zip
!unzip -q dataset.zip -d dataset

--2021-05-19 18:47:50--  https://market1501.s3-us-west-2.amazonaws.com/dataset.zip
Resolving market1501.s3-us-west-2.amazonaws.com (market1501.s3-us-west-2.amazonaws.com)... 52.218.197.153
Connecting to market1501.s3-us-west-2.amazonaws.com (market1501.s3-us-west-2.amazonaws.com)|52.218.197.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82925180 (79M) [application/zip]
Saving to: ‘dataset.zip’

dataset.zip         100%[===================>]  79.08M  28.1MB/s    in 2.8s    

2021-05-19 18:47:54 (28.1 MB/s) - ‘dataset.zip’ saved [82925180/82925180]



In [2]:
!rm -rf /content/deeplearning_unitn
!git clone https://github.com/deayalar/deeplearning_unitn.git

Cloning into 'deeplearning_unitn'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 85 (delta 37), reused 63 (delta 19), pack-reused 0
Unpacking objects: 100% (85/85), done.


In [ ]:
!nvidia-smi

Tue May 18 17:50:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
config = dict(
    wandb = False,
    device = "auto", # Select an specific device None to select automatically
    #train_root = "/content/dataset/train",
    #attributes_file = "/content/dataset/annotations_train.csv"
    train_root = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/train",
    test_root = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/test",
    queries_root = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/queries",
    attributes_file = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/annotations_train.csv",
    dataset="Market1501",
    backbone = "resnet18",
    loss = "triplet / crossentropy, etc",
    split = dict(
        full_training_size = 0.75,
        train_size = 0.8
    ),
    compose = dict(
        resize_h = 224,
        resize_w = 224
    ),
    epochs=1,
    training_batch_size=128,
    learning_rate=0.01,
    weight_decay=0.000001, 
    momentum=0.9, 
    mAP_rank=10,
    architecture="LeNet")

In [10]:
%cd /content/deeplearning_unitn

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

import cost_functions
from evaluation import Evaluator
from my_datasets.reid_dataset import Market1501
from utils.split_data import ValidationSplitter, TrainingSplitter
from models.reid_model import ReIdModel, FinetunedModel
from sklearn.model_selection import train_test_split

if config["device"] == "auto":
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
else:
    device = config["device"]
print(device)

[Errno 2] No such file or directory: '/content/deeplearning_unitn'
/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dl_unitn
cuda:0


In [3]:
!pip install wandb -q
import wandb
wandb.login()

/bin/bash: pip: command not found


ModuleNotFoundError: No module named 'wandb'

In [3]:
def model_pipeline(hyperparameters):
    """
    This function corresponds to the basic pipeline of all tested models
    1) Setup based on the configuration
    2) Train the model
    3) Test performance
    """
    config = hyperparameters
    if config["wandb"]:
      wandb.init(entity="dl_unitn", project="dl_project", config=hyperparameters)
      config = wandb.config
    print(config)
    
    model, train_loader, val_loader, val_queries_loader, id_ground_truth_dict, criterion, optimizer = setup(config)
    print(model)

    train(model, train_loader, val_loader, criterion, optimizer, config)

    test(model, val_loader, val_queries_loader, config, save_model=True) #True to save model

    return model

In [5]:
def build_ground_truth(val_set, val_queries):
    values = []
    for idx_q, q in enumerate(val_queries):
        matches = []
        for idx_v, v in enumerate(val_set):
            if v.split("_")[0] == q.split("_")[0]:
                matches.append(idx_v)
        value = set(matches)
        values.append(value)
        
    ground_truth_dict = dict(zip(list(range(0, len(val_queries))), values))

    #values = []
    #for q in val_queries:
    #    matches = []
    #    for v in val_set:
    #        if v.split("_")[0] == q.split("_")[0]:
    #            matches.append(v)
    #    value = set(matches)
    #    values.append(value)
    #ground_truth_dict = dict(zip(val_queries, values))

    return ground_truth_dict


In [11]:
def setup(config):
    # This validation set is used to estimate the performance on the final test set
    splitter = ValidationSplitter(train_root=config["train_root"], 
                                  test_root=config["test_root"], 
                                  queries_root=config["queries_root"])
    train_set, val_set, val_queries = splitter.split(train_size=config["split"]["full_training_size"],
                                                     random_seed=42)
    #Create pytorch Datasets 
    composed = transforms.Compose([transforms.Resize((config["compose"]["resize_h"], 
                                                      config["compose"]["resize_w"])),
                                   transforms.ToTensor(),
                                   transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                        std=[0.229, 0.224, 0.225])])
    
    train_dataset = Market1501(root_dir=config["train_root"],
                            attributes_file=config["attributes_file"],
                            #full_train_set=full_train_set,
                            images_list=train_set,
                            transform=composed)
                            
    val_dataset = Market1501(root_dir=config["train_root"],
                         attributes_file=config["attributes_file"],
                         #full_train_set = full_train_set,
                         images_list=val_set,
                         transform=composed)

    val_queries_dataset = Market1501(root_dir=config["train_root"],
                         attributes_file=config["attributes_file"],
                         #full_train_set = full_train_set,
                         images_list=val_queries,
                         transform=composed)

    train_loader = torch.utils.data.DataLoader(train_dataset, 
                                               batch_size=config["training_batch_size"], 
                                               shuffle=True, 
                                               num_workers=8)
                                               
    #We need to load the entire validation set and queries in one batch
    val_loader = torch.utils.data.DataLoader(val_dataset, 
                                             #batch_size=len(val_dataset), 
                                             batch_size=16, 
                                             shuffle=False, 
                                             num_workers=8)

    val_queries_loader = torch.utils.data.DataLoader(val_queries_dataset, 
                                             #batch_size=len(val_queries_dataset), 
                                             batch_size=16, 
                                             shuffle=False, 
                                             num_workers=8)

    id_ground_truth_dict = build_ground_truth(val_set, val_queries)

    #model = ReIdModel(n_classes=len(train_dataset.classes)).to(device)
    attr_len = len(train_dataset[0][2]) #Number of attributes in the csv: 27
    print(f"Number of attributes: {attr_len}")
    model = FinetunedModel(architecture=config["backbone"] ,n_classes=attr_len).to(device)

    # Setup the optimization problem
    criterion = cost_functions.cross_entropy()
    optimizer = torch.optim.SGD(model.parameters(), 
                                lr=config["learning_rate"], 
                                weight_decay=config["weight_decay"], 
                                momentum=config["momentum"])
    
    return model, train_loader, val_loader, val_queries_loader, id_ground_truth_dict, criterion, optimizer

In [5]:
def train(model, train_loader, val_loader, criterion, optimizer, config):
    # tell wandb to watch what the model gets up to: gradients, weights, and more!
    if config["wandb"]:
        wandb.watch(model, criterion, log="all", log_freq=10)
    model.train()
    # Run training and track with wandb
    total_batches = len(train_loader) * config["epochs"]
    example_ct = 0  # number of seen examples
    batch_ct = 0

    for epoch in tqdm(range(config["epochs"])):
        for batch_idx, (inputs, labels, attributes) in enumerate(train_loader):
            loss = train_batch(inputs, labels, model, optimizer, criterion)

            example_ct +=  len(inputs)
            batch_ct += 1

            if ((batch_ct + 1) % 50) == 0:
                train_log(loss, example_ct, epoch)


def train_batch(inputs, labels, model, optimizer, criterion):
    inputs, labels = inputs.to(device), labels.to(device)
    
    # Forward pass
    outputs = model(inputs)
    # Apply the loss
    loss = criterion(outputs, labels)
    
    optimizer.zero_grad()
    # Backward pass
    loss.backward()

    # Step with optimizer
    optimizer.step()

    # Zeros the gradient

    return loss

In [8]:
def train_log(loss, example_ct, epoch):
    loss = float(loss)
    if config["wandb"]:
        wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
    print(f"Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")

In [9]:
# This coulf be the attributes classification function
def test(model, loader, config, save_model=False):
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        correct, total = 0, 0
        for inputs, labels, attributes in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f"Accuracy of the model on the {total} " +
              f"test images: {100 * correct / total}%")
        if config["wandb"]:
            wandb.log({"test_accuracy": correct / total})
    if save_model :
      # Save the model in the exchangeable ONNX format
      torch.onnx.export(model, inputs, "model.onnx")
      if config["wandb"]:
        wandb.save("model.onnx")

In [5]:
# Build, train and analyze the model with the pipeline
model = model_pipeline(config)

NameError: name 'model_pipeline' is not defined

In [12]:
model, train_loader, val_loader, val_queries_loader, id_ground_truth_dict, criterion, optimizer = setup(config)

Extract queries proportion: 0.11423344682148483
Identities in train set: 563
Identities in validation set: 188
Train set size: 9593
Validation set size: 3008
Number of validation queries: 388
Number of attributes: 27
Backbone feature size: 512


In [13]:
def test_mAP(model, gallery_loader, queries_loader, ground_truth_dict, config, save_model=False):
    """
    This function returns the mAP performance of a given model 
    Params:
    model: model to be evaluated
    gallery: tensor that contains the feature representations of the target images in validation or test set
    queries: tensor that contains feature representations of the queries
    rank: top number of elements to retrieve

    Returns:
    mAP performance of the model
    """
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        correct, total = 0, 0
        
        # Compute the features for queries and gallery

        gallery, gallery_labels, gallery_attr = next(iter(gallery_loader))
        gallery = gallery.to(device)
        print(gallery.size())
        gallery_features = model(gallery, get_features=True)
        print(gallery_features.size())
        
        queries, query_labels, query_attr = next(iter(queries_loader))
        queries = queries.to(device)
        print(queries.size())
        query_features = model(queries, get_features=True)
        print(query_features.size())
        
        # Build the cosine similarity matrix between the all the queries and all the elements in gallery
        # cos = nn.CosineSimilarity()
        sims_matrix = torch.empty(query_features.size()[0], gallery_features.size()[0]).to(device)
        for idx, q in enumerate(query_features):
            #print(q.get_device())
            #print(validation_features.get_device())
            sims_matrix[idx] = F.cosine_similarity(q, gallery_features, dim=-1)

        print(sims_matrix.size())
        sorted_index = torch.argsort(sims_matrix, dim=1, descending=True)
        top_k = sorted_index.narrow_copy(dim=1, start=0, length=config["mAP_rank"])

        #Build the dictionary to compute the mAP
        predictions_dict = {idx:  r for idx, r in enumerate(top_k.tolist())}
        mAP = Evaluator.evaluate_map(predictions_dict, ground_truth_dict)
        
        print(f"mAP: {mAP}")
        if config["wandb"]:
            wandb.log({"mAP": mAP})
    if save_model :
      # Save the model in the exchangeable ONNX format
      torch.onnx.export(model, inputs, "model.onnx", export_params=True)
      if config["wandb"]:
        wandb.save("model.onnx")

In [14]:
test_mAP(model, val_loader, val_queries_loader, id_ground_truth_dict, config, save_model=False)

torch.Size([16, 3, 224, 224])
torch.Size([16, 512])
torch.Size([388, 3, 224, 224])


RuntimeError: CUDA out of memory. Tried to allocate 1.16 GiB (GPU 0; 1.96 GiB total capacity; 321.50 MiB already allocated; 637.44 MiB free; 332.00 MiB reserved in total by PyTorch)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class IdentificationModel(nn.Module):
    """
    Model trained to produce a vector representation of the image
    input_size: size of the input vector -> features size + attributes size
    embedding_size: size of the output vector
    """
    # So far it is a simple fully connected network
    def __init__(self, input_size, embedding_size):
        super().__init__()
        self.feature_extractor = nn.Sequential(
            nn.Linear(in_features=input_size, out_features=512),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=embedding_size),
            nn.Flatten()
        )


    def forward(self, x):
        x = self.feature_extractor(x)
        return x